In [ ]:
import os
import pandas as pd
import networkx as nx


PATH = os.path.abspath("")
PATH

In [9]:
distances = pd.read_csv(os.path.join(PATH,'./database/matriz-paa.csv'),
                        sep=';',
                        index_col='Unnamed: 0')
distances = distances.applymap(lambda x: int(x.replace(' Km', '')))
distances


,América-MG,Athletico-PR,Atlético-MG,Bahia,Botafogo,Bragantino,Corinthians,Coritiba,Cruzeiro,Cuiabá,Flamengo,Fluminense,Fortaleza,Goiás,Grêmio,Internacional,Palmeiras,Santos,São Paulo,Vasco
América-MG,0,821,1,966,349,423,493,821,1,1374,349,349,1895,667,1343,1343,493,513,493,349
Athletico-PR,821,0,821,1788,674,392,337,1,821,1304,674,674,2673,975,546,546,337,340,337,674
Atlético-MG,1,821,0,966,349,423,493,821,1,1374,349,349,1895,667,1343,1343,493,513,493,349
Bahia,966,1788,966,0,1212,1398,1458,1788,966,1918,1212,1212,1029,1227,2306,2306,1458,1474,1458,1212
Botafogo,349,674,349,1212,0,343,360,674,349,1577,1,1,2192,938,1125,1125,360,340,360,1
Bragantino,430,392,430,1398,343,0,69,392,430,1293,343,343,2306,753,916,916,69,115,69,343
Corinthians,491,338,491,1458,360,69,0,338,491,1327,360,360,2373,813,851,851,1,56,1,360
Coritiba,821,1,821,1788,674,392,337,0,821,1304,674,674,2673,975,546,546,337,340,337,674
Cruzeiro,1,821,1,966,349,423,493,821,0,1374,349,349,1895,667,1343,1343,493,513,493,349
Cuiabá,1375,1304,1375,1918,1582,1294,1327,1304,1375,0,1582,1582,2331,741,1681,1681,1327,1381,1327,1582


In [91]:
n = distances.shape[0]         

G = nx.complete_graph(n)
graph_names = dict(zip(list(G.nodes), distances.index))
G = nx.relabel_nodes(G, graph_names)
attrs_g = {'total_matches': 19, 
           'max_home_matches': 10,
           'max_away_matches': 10,
           'num_home_matches': 0,
           'num_away_matches': 0,
           'max_consecutive_match_type':2}
G.graph.update(attrs_g)


nx.set_edge_attributes(G, {(u, v): {'match_type': None} for u, v in G.edges})
for i,j in G.edges:
    G.edges[i,j]['length'] = distances.loc[i,j]

In [112]:
tour = ['Fortaleza']
while len(tour) < n:
    i = tour[-1]   
    min_length = min( G.edges[i,j]['length'] for j in G.neighbors(i) 
                     if j not in tour )
    nearest_neighbors = [ j for j in G.neighbors(i) 
                         if j not in tour and G.edges[i,j]['length'] == min_length ]
    tour.append( nearest_neighbors[0] )

tour_edges = [ (tour[i-1],tour[i], G.edges.get((tour[i-1],tour[i]))['length']) for i in range(n) ]





In [113]:
tour_edges

[('Cuiabá', 'Fortaleza', 2331),
 ('Fortaleza', 'Bahia', 1029),
 ('Bahia', 'América-MG', 966),
 ('América-MG', 'Atlético-MG', 1),
 ('Atlético-MG', 'Cruzeiro', 1),
 ('Cruzeiro', 'Botafogo', 349),
 ('Botafogo', 'Flamengo', 1),
 ('Flamengo', 'Fluminense', 1),
 ('Fluminense', 'Vasco', 1),
 ('Vasco', 'Santos', 338),
 ('Santos', 'Corinthians', 56),
 ('Corinthians', 'Palmeiras', 1),
 ('Palmeiras', 'São Paulo', 1),
 ('São Paulo', 'Bragantino', 69),
 ('Bragantino', 'Athletico-PR', 392),
 ('Athletico-PR', 'Coritiba', 1),
 ('Coritiba', 'Grêmio', 546),
 ('Grêmio', 'Internacional', 1),
 ('Internacional', 'Goiás', 1499),
 ('Goiás', 'Cuiabá', 741)]

In [114]:
sum(t[2] for t in tour_edges)

8325

In [109]:
nx.path_weight(G,tour,'length')

5994

In [75]:
def set_match_type(is_home):
    match_types = ('home','away')
    return match_types[0] if is_home else match_types[1]

In [92]:
attrs_g = {'total_matches': 19, 
           'max_home_matches': 10,
           'max_away_matches': 10,
           'num_home_matches': 0,
           'num_away_matches': 0,
           'max_consecutive_match_type':2}
G.graph.update(attrs_g)
home = 'Fortaleza'
tour = [home] # é necessário estabelecer o inicio
matches = []
is_home = True
while len(tour) <= G.graph['total_matches']:
    if is_home: # if is a home match
        i = home
        max_length = max(G.edges[i,j]['length'] for j in G.neighbors(i) if j not in tour)
        next_match = [j for j in G.neighbors(i) 
                        if j not in tour and G.edges[i,j]['length'] == max_length][0]
        G.graph['num_home_matches'] += 1

    else:
        min_length = min(G.edges[i,j]['length'] for j in G.neighbors(i) if j not in tour)
        next_match = [j for j in G.neighbors(i) 
                        if j not in tour and G.edges[i,j]['length'] == min_length][0]
        G.graph['num_away_matches'] += 1
    
    G.edges.get((i, next_match))['match_type'] = set_match_type(is_home)
    matches.append((i, next_match))
    tour.append(next_match)
    i = tour[-1]   
    if (is_home 
        and G.graph['num_home_matches'] % G.graph['max_consecutive_match_type'] == 0):
        is_home = not is_home
        i = home
        continue
    if(not is_home 
        and G.graph['num_away_matches'] % G.graph['max_consecutive_match_type'] == 0):
        is_home = not is_home
        continue
tour_edges = [ (tour[i-1],tour[i]) for i in range(n) ]

Fortaleza Grêmio 3217
Fortaleza Internacional 3217
Fortaleza Bahia 1029
Bahia América-MG 966
Fortaleza Athletico-PR 2673
Fortaleza Coritiba 2673
Fortaleza Goiás 1857
Goiás Atlético-MG 667
Fortaleza Santos 2401
Fortaleza Corinthians 2373
Fortaleza Cruzeiro 1895
Cruzeiro Botafogo 349
Fortaleza Palmeiras 2373
Fortaleza São Paulo 2373
Fortaleza Flamengo 2192
Flamengo Fluminense 1
Fortaleza Cuiabá 2331
Fortaleza Bragantino 2306
Fortaleza Vasco 2200


In [97]:
matches

[('Fortaleza', 'Grêmio'),
 ('Fortaleza', 'Internacional'),
 ('Fortaleza', 'Bahia'),
 ('Bahia', 'América-MG'),
 ('Fortaleza', 'Athletico-PR'),
 ('Fortaleza', 'Coritiba'),
 ('Fortaleza', 'Goiás'),
 ('Goiás', 'Atlético-MG'),
 ('Fortaleza', 'Santos'),
 ('Fortaleza', 'Corinthians'),
 ('Fortaleza', 'Cruzeiro'),
 ('Cruzeiro', 'Botafogo'),
 ('Fortaleza', 'Palmeiras'),
 ('Fortaleza', 'São Paulo'),
 ('Fortaleza', 'Flamengo'),
 ('Flamengo', 'Fluminense'),
 ('Fortaleza', 'Cuiabá'),
 ('Fortaleza', 'Bragantino'),
 ('Fortaleza', 'Vasco')]

In [103]:
list_of_matches = []
for mat in matches:
    if G.edges.get(mat)['match_type'] == 'home':
        list_of_matches.append((*mat, 0))
    else:
        if mat[0] == 'Fortaleza':
            list_of_matches.append((*mat, G.edges.get(mat)['length']))
        else:
            dist = G.edges.get(mat)['length'] + G.edges.get((mat[1], 'Fortaleza'))['length']
            list_of_matches.append(('Fortaleza', mat[1], dist))

list_of_matches

[('Fortaleza', 'Grêmio', 0),
 ('Fortaleza', 'Internacional', 0),
 ('Fortaleza', 'Bahia', 1029),
 ('Fortaleza', 'América-MG', 2861),
 ('Fortaleza', 'Athletico-PR', 0),
 ('Fortaleza', 'Coritiba', 0),
 ('Fortaleza', 'Goiás', 1857),
 ('Fortaleza', 'Atlético-MG', 2562),
 ('Fortaleza', 'Santos', 0),
 ('Fortaleza', 'Corinthians', 0),
 ('Fortaleza', 'Cruzeiro', 1895),
 ('Fortaleza', 'Botafogo', 2541),
 ('Fortaleza', 'Palmeiras', 0),
 ('Fortaleza', 'São Paulo', 0),
 ('Fortaleza', 'Flamengo', 2192),
 ('Fortaleza', 'Fluminense', 2193),
 ('Fortaleza', 'Cuiabá', 0),
 ('Fortaleza', 'Bragantino', 0),
 ('Fortaleza', 'Vasco', 2200)]

In [1]:
print("Fortaleza viajou: 19330 km")

Fortaleza viajou: 19330 km
